In [4]:
import pickle
import os

In [5]:
path_gt = "/home/ybouaouni/workspace/Training/PII-Data-Detection-Transformers/data/holdout/validation.csv"

In [29]:
def load_pickle_files(paths):
    """
    Load pickle files from a list of paths.

    Args:
    paths (list of str): List of file paths.

    Returns:
    list: List of opened pickle files.
    """
    pickle_files = []
    for path in paths:
        with open(path, 'rb') as file:
            pickle_files.append(pickle.load(file))
    return pickle_files

def pickle2preds(pickle_files):

    preds_ensemble, pred_class_ensemble, ds_dict_ensemble, id2label_ensemble  = [], [], [], []
    for pickle_file in pickle_files:
        preds_ensemble.append(pickle_file['preds'])
        pred_class_ensemble.append(pickle_file['pred_classes'])
        ds_dict_ensemble.append(pickle_file['ds_dict'])
        id2label_ensemble.append(pickle_file['id2label'])
    return {"preds": preds_ensemble , "pred_class": pred_class_ensemble, "id2label": id2label_ensemble , "ds_dict": ds_dict_ensemble}
        

root = "/home/ybouaouni/workspace/Training/PII-Data-Detection-Transformers/piidetect/Deberta-Base-HPO-Preds"
models = [
"deberta_base_ep2_bs4_acc1_lr2e-05_dropout0.15_tonyarobertson",
"deberta_base_ep2_bs6_acc1_lr2e-05_dropout0.1_mpware",
"deberta_base_ep2_bs4_acc1_lr2e-05_dropout0.1_nicholas",
"deberta_base_ep2_bs4_acc1_lr2e-05_dropout0.1_tonyarobertson"
]


In [30]:

model_paths = [os.path.join(root, model, "predictions.pickle") for model in models]
pickle_results = load_pickle_files(model_paths)

# Average predictions

In [31]:
from utils.utils_test import *

THRESHOLD = 0.9

In [32]:
preds = [ pickle_result['preds'] for pickle_result in pickle_results]
ds_dict = pickle_results[0]['ds_dict']
id2label = pickle_results[0]['id2label']

In [33]:
import numpy as np
from collections import Counter
# Combine the predictions into a single list of tuples
combined_preds = zip(*preds)

# Perform max voting for each position
max_voted_preds = []
for positions in combined_preds:
    curr_sequence = []
    for i in range(len(positions[0][0])):
        curr_sequence.append(Counter([positions[0][0][i], positions[1][0][i], positions[2][0][i]]).most_common(1)[0][0])
    max_voted_preds.append(np.array([np.array(curr_sequence)]))
# Print the max voted predictions
preds_final = max_voted_preds



In [34]:


ignored_labels = "O"
processed, pairs = get_doc_token_pred_triplets(
    preds_final, Dataset.from_dict(ds_dict), id2label, ignored_labels
)


df = pd.DataFrame(processed)
df["row_id"] = list(range(len(df)))
df_gt = pd.read_csv(path_gt)
precision, recall, f1 = Evaluator.compute_metrics_eval(df, df_gt)
print(precision, recall, f1)


True Positives: 522
False Positives: 130
False Negatives: 10
Precision: 0.8006134969325154
Recall: 0.981203007518797
f1 competition: 0.9727637614678899
0.8006134969325154 0.981203007518797 0.9727637614678899
